# QAOA method to solve for MIS Problem
### This notebook explores the effect of the QAOA and how it performs on increasingly more complex MIS graphs and tunes parameters and pulse sequences!

In [ ]:
using Bloqade
using Random
using Optim
Random.seed!(42)
using PythonCall
plt = pyimport("matplotlib.pyplot");

In [ ]:
atom1 = AtomList([(0.0, 0.0), (0.0, 4.5), (0.0, 9.0), (0.0, 13.5), (0.0, 18.0)], [(4.5, 0.0), (4.5, 4.5), 
    (4.5, 9.0), (4.5, 13.5), (4.5, 4.5)], )